# Попробуем выжать все из простых фичей

# Загрузка данных

In [1]:
import pandas as pd
import json

In [ ]:
def open_dict(d):
    for k, v in d.items():
        if type(v) is dict:
            for _k, _v in open_dict(v):
                yield _k, _v
        else:
            yield k, v

In [ ]:
def parse_line(l):
    x = json.loads(l)
    return dict(open_dict(x))

In [ ]:
%%time
with open("../data/train.json", "r") as f:
    pd_train = pd.DataFrame([parse_line(l) for l in f.readlines()])

In [ ]:
%%time
with open("../data/test.json", "r") as f:
    pd_test = pd.DataFrame([parse_line(l) for l in f.readlines()])

In [ ]:
%%time
pd_train.to_pickle("../tdata/pd_train.pickle")
pd_test.to_pickle("../tdata/pd_test.pickle")

# Обработка

In [2]:
import pickle

In [3]:
%%time
pd_train = pickle.load(open("../tdata/pd_train.pickle", "rb"))
pd_test = pickle.load(open("../tdata/pd_test.pickle", "rb"))
pd_data = pd.concat([pd_train, pd_test], axis=0, keys=["train", "test"])
del pd_train, pd_test

CPU times: user 6.5 s, sys: 1.9 s, total: 8.4 s
Wall time: 8.68 s


## Функции

In [4]:
def del_column_if_exist(data, column):
    if column in data.columns:
        del data[column]

In [60]:
def date_to_f(data):
    if "timestamp" not in data.columns:
        dt = pd.to_datetime(data['$date'])
        data["timestamp"] = dt.apply(lambda x: x.timestamp())
        data["ts_year"] = dt.apply(lambda x: x.year)
        data["ts_month"] = dt.apply(lambda x: x.month)
        # del data['$date']

In [6]:
def del_unused(data):
    for column in pd_data.columns[pd_data.columns.str.contains("url")]:
        del_column_if_exist(data, column)
    for column in ["_timestamp", "al:windows_phone:app_id", "alternate",
                   "apple-touch-icon-precomposed", "canonical", "yandex-verification", "post_id",
                   "al:android:package", "al:windows_phone:app_name", "apple-mobile-web-app-title",
                  "image_src", "icon", "msapplication-TileColor", "msapplication-TileImage", "og:image",
                  "flow", "og:type", "pocket-site-verification", "polling", "referrer", "stylesheet",
                  "twitter:card", "viewport", "fb:app_id"]:
        del_column_if_exist(data, column)

In [7]:
def fill_robots(data):
    data.robots.fillna("no_robots", inplace = True)

## Обработка данных

In [61]:
date_to_f(pd_data)

In [9]:
del_unused(pd_data)

In [10]:
fill_robots(pd_data)

In [59]:
pd_data

$date  \
train 0      2006-07-15T01:48:00.000Z   
      1      2006-07-15T14:23:00.000Z   
      2      2006-07-04T01:54:00.000Z   
      3      2007-01-11T03:37:00.000Z   
      4      2007-07-30T16:18:00.000Z   
      5      2013-02-09T18:02:00.000Z   
      6      2013-02-07T14:06:00.000Z   
      7      2013-02-07T14:11:00.000Z   
      8      2013-02-07T14:39:00.000Z   
      9      2013-02-07T15:26:00.000Z   
      10     2013-02-07T16:34:00.000Z   
      11     2013-02-12T15:48:00.000Z   
      12     2013-02-07T16:33:00.000Z   
      13     2013-02-07T16:09:00.000Z   
      14     2013-02-07T17:02:00.000Z   
      15     2007-07-30T16:53:00.000Z   
      16     2013-02-07T16:55:00.000Z   
      17     2013-02-07T17:50:00.000Z   
      18     2013-02-07T18:05:00.000Z   
      19     2013-02-07T17:03:00.000Z   
      20     2013-02-07T18:18:00.000Z   
      21     2013-02-11T17:41:00.000Z   
      22     2013-02-07T19:02:00.000Z   
      23     2013-02-08T15:42:00.000Z   
      24     2013-02-07T20:02:00.000Z   
      25     2013-02-23T20:35:00.000Z   
      26     2007-07-30T16:29:00.000Z   
      27     2013-02-07T22:23:00.000Z   
      28     2013-02-07T21:30:00.000Z   
      29     2013-02-07T23:31:00.000Z   
...                               ...   
test  52883  2013-02-06T20:43:00.000Z   
      52884  2013-02-06T19:32:00.000Z   
      52885  2013-02-06T20:03:00.000Z   
      52886  2013-02-06T21:32:00.000Z   
      52887  2013-02-07T10:30:00.000Z   
      52888  2013-02-06T21:42:00.000Z   
      52889  2013-02-07T22:08:00.000Z   
      52890  2013-02-06T23:58:00.000Z   
      52891  2007-07-30T15:09:00.000Z   
      52892  2013-02-09T21:56:00.000Z   
      52893  2013-02-07T07:55:00.000Z   
      52894  2013-02-07T12:54:00.000Z   
      52895  2013-02-08T10:40:00.000Z   
      52896  2013-02-07T11:20:00.000Z   
      52897  2013-02-07T10:29:00.000Z   
      52898  2013-02-07T11:10:00.000Z   
      52899  2013-02-07T08:16:00.000Z   
      52900  2013-02-07T11:34:00.000Z   
      52901  2013-02-15T05:27:00.000Z   
      52902  2007-07-30T16:37:00.000Z   
      52903  2013-02-07T15:11:00.000Z   
      52904  2013-02-07T14:05:00.000Z   
      52905  2013-02-12T12:04:00.000Z   
      52906  2013-02-07T12:24:00.000Z   
      52907  2013-02-07T12:26:00.000Z   
      52908  2013-02-07T19:12:00.000Z   
      52909  2013-02-07T12:06:00.000Z   
      52910  2013-02-07T13:57:00.000Z   
      52911  2013-02-07T13:32:00.000Z   
      52912  2013-02-07T12:57:00.000Z   

                                                           _id  \
train 0                           https://habrahabr.ru/post/7/   
      1                          https://geektimes.ru/post/11/   
      2                         https://geektimes.ru/post/112/   
      3                        https://geektimes.ru/post/1127/   
      4                       https://geektimes.ru/post/12664/   
      5                      https://geektimes.ru/post/168615/   
      6                      https://habrahabr.ru/post/168617/   
      7                      https://geektimes.ru/post/257768/   
      8                      https://habrahabr.ru/post/168625/   
      9                      https://habrahabr.ru/post/168631/   
      10                     https://geektimes.ru/post/168633/   
      11     https://habrahabr.ru/company/sonyxperia/blog/1...   
      12                     https://habrahabr.ru/post/168637/   
      13                     https://habrahabr.ru/post/168635/   
      14                     https://habrahabr.ru/post/168641/   
      15                      https://geektimes.ru/post/12667/   
      16                     https://geektimes.ru/post/168639/   
      17      https://habrahabr.ru/company/ruward/blog/168647/   
      18                     https://geektimes.ru/post/168645/   
      19                     https://habrahabr.ru/post/168643/   
      20     https://habrahabr.ru/company/etnasoft/blog/168...   
      21       https://habrahabr.ru/c

# Оставляем тексовые фичи

In [12]:
pd_data[["content", "description", "flags", "hubs", "keywords", "tags", "title"]]

content  \
train 0      У меня такое ощущение, что logout время от вре...   
      1      Я вот тут подумал, смотря, скажем, на <a href=...   
      2      Да! Действительно утром зашел. И опустил карму...   
      3      Ну, не прошло и нескольких дней, как Cisco под...   
      4      На прошлой неделе в сети то тут то там появлял...   
      5      Очень полезное приложение обнаружилось недавно...   
      6      <img src="https://habrastorage.org/storage2/ee...   
      7      В конце позапрошлого года <a href="http://geek...   
      8      <img align="right" src="https://habrastorage.o...   
      9      <img src="https://habrastorage.org/storage2/be...   
      10     <h4>Преамбула утилизации<br>\r\n</h4>Вопрос ут...   
      11     <img src="https://habrastorage.org/storage2/39...   
      12     <img src="https://habrastorage.org/storage2/36...   
      13     Без синхронизаций и т.д., в варианте «есть хос...   
      14     На сегодняшний день всем хорошо знакомы такие ...   
      15     По заявлению компании Apple, срок «жизни» акку...   
      16     Взялся писать этот пост спонтанно, под действи...   
      17     <a href="http://ruward.ru"><img src="https://h...   
      18     Почему я говорю так? Потому что я на пути созд...   
      19     Бытует противоречивое мнение, что на проекте о...   
      20     Я занимаюсь созданием высоконагруженных прилож...   
      21     <img src="https://habrastorage.org/storage2/44...   
      22     Мы рады сообщить, что вчера вышла новая версия...   
      23     Ролики с демонстрацией пиксельной подсветки вы...   
      24     <img src="https://habrastorage.org/getpro/geek...   
      25     Как показали комментарии к <a href="http://hab...   
      26     Военная индустрия знает немало способов и мето...   
      27     По данной теме очень много холиваров и разрозн...   
      28     С выходом .NET Framework 4.0 в состав <a href=...   
      29     Не так давно вышел новый Qt, обещающий большое...   
...                                                        ...   
test  52883  Близится тот волнительный миг, когда 23 феврал...   
      52884  Как правило, разработка и отладка приложения в...   
      52885  Все-таки Curiosity хипстер и, скорее, все же д...   
      52886  Посмотрим на код:<br>\r\n<br>\r\n<pre><code cl...   
      52887  <img src="https://habrastorage.org/getpro/habr...   
      52888  <img src="http://www.wired.com/design/wp-conte...   
      52889  Аппаратная организация VPN в РФ в основном дер...   
      52890  Есть одна важная для меня RSS-лента. И хочется...   
      52891  Блог TUAW <a href="http://www.tuaw.com/2007/07...   
      52892  Недавний пост о выходе Kotin M5 вызвал у меня ...   
      52893  <img src="https://habrastorage.org/getpro/habr...   
      52894  Каждый, наверное, сталкивался с сайтами, предл...   
      52895  <a href="http://img40.imageshack.us/img40/7047...   
      52896  <img src="https://habrastorage.org/storage2/7c...   
      52897  <img src="https://habrastorage.org/storage2/35...   
      52898  Иногда находится интересная вакансия, но о ком...   
      52899  <iframe width="560" height="315" src="//www.yo...   
      52900  Кроссворд, подсказки в котором — регулярные вы...   
      52901  В <a href="http://habrahabr.ru/post/166361/">п...   
      52902  Компания Microsoft разработала и выложила на с...   
      52903  Как известно, оптимизатор запросов SQL Server,...   
      52904  В этой статье мы подготовили для вас подробную...   
      52905  Мне хотелось создать <s>прекрасный</s> объемлю...   
      52906  <img src="https://habrastorage.org/getpro/geek...   
      52907  В данном обзоре речь пойдет о модели mini FLOW...   
      52908  Я знаю на Хабре скептично относятся к Вконтакт...   
      52909  <img src="https://habrastorage.org/storage2/b0...   
      52910  Это первый пост из <a href="http://habrahabr.r...   
      52911  Как <a href="http://habrahabr.ru/post/165737/"...   
      52912  Сетевой инженер 

# Split

In [13]:
import numpy as np

In [14]:
ix_test = np.isin(pd_data.loc["train"].index, np.random.choice(pd_data.loc["train"].index, 12000, replace=False))
ix_train = ~ix_test

# Фичи не из текста

## Feature_list

In [15]:
d_features = dict()

# domain

In [16]:
from sklearn.preprocessing import LabelBinarizer

In [17]:
lb_domain = LabelBinarizer()
lb_domain.fit(pd_data.loc["train"][ix_train]["domain"])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [18]:
lb_domain.classes_

array(['geektimes.ru', 'habrahabr.ru'],
      dtype='<U12')

In [19]:
d_features["domains"] = (lb_domain.transform(pd_data.loc["train"][ix_train]["domain"]),
                         lb_domain.transform(pd_data.loc["train"][ix_test]["domain"]),
                         lb_domain.transform(pd_data.loc["test"]["domain"]))

## robots

In [20]:
lb_robots = LabelBinarizer()
lb_robots.fit(pd_data.loc["train"][ix_train]["robots"])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [21]:
lb_robots.classes_

array(['no_robots', 'noindex'],
      dtype='<U9')

In [22]:
d_features["robots"] = (lb_robots.transform(pd_data.loc["train"][ix_train]["robots"]),
                    lb_robots.transform(pd_data.loc["train"][ix_test]["robots"]),
                    lb_robots.transform(pd_data.loc["test"]["robots"]))

In [95]:
pickle.dump(lb_robots.transform(pd_data.loc["train"]["robots"]), open("../features/robots_train.pickle", "wb"))
pickle.dump(lb_robots.transform(pd_data.loc["test"]["robots"]), open("../features/robots_test.pickle", "wb"))

## Timestamp

In [23]:
d_features["timestamp"] = (pd_data.loc["train"][ix_train]["timestamp"].values.reshape(-1,1),
                           pd_data.loc["train"][ix_test]["timestamp"].values.reshape(-1,1),
                           pd_data.loc["test"]["timestamp"].values.reshape(-1,1))

## lb_year

In [24]:
lb_year = LabelBinarizer()
lb_year.fit(pd_data.loc["train"][ix_train]["ts_year"])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [25]:
lb_year.classes_

array([2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016])

In [50]:
d_features["lb_year"] = (lb_year.transform(pd_data.loc["train"][ix_train]["ts_year"]),
                         lb_year.transform(pd_data.loc["train"][ix_test]["ts_year"]),
                         lb_year.transform(pd_data.loc["test"]["ts_year"]))

## lb_month

In [62]:
lb_month = LabelBinarizer()
lb_month.fit(pd_data.loc["train"][ix_train]["ts_month"])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [63]:
lb_month.classes_

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [64]:
d_features["lb_month"] = (lb_month.transform(pd_data.loc["train"][ix_train]["ts_month"]),
                         lb_month.transform(pd_data.loc["train"][ix_test]["ts_month"]),
                         lb_month.transform(pd_data.loc["test"]["ts_month"]))

## author_rating

In [51]:
s_author_rating = pickle.load(open("../features/author_rating.pickle", "rb"))

In [52]:
d_features["author_rating"] = (np.log(s_author_rating.loc["train"][ix_train].values+1).reshape(-1,1),
                         np.log(s_author_rating.loc["train"][ix_test].values+1).reshape(-1,1),
                         np.log(s_author_rating.loc["test"].values+1).reshape(-1,1))

## Flags

In [77]:
s_flags = pd_data.flags.apply(lambda x: " ".join([i.replace(" ", "_") for i in x]))

In [75]:
from sklearn.feature_extraction.text import CountVectorizer

In [76]:
cv = CountVectorizer()

In [78]:
cv.fit(s_flags.loc["train"][ix_train].values)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [79]:
cv.tr

{'news': 0,
 'recovery': 1,
 'rss_feed': 2,
 'sandbox': 3,
 'translation': 4,
 'tutorial': 5}

In [91]:
d_features["flags"] = (cv.transform(s_flags.loc["train"][ix_train]).toarray(),
                       cv.transform(s_flags.loc["train"][ix_test]).toarray(),
                       cv.transform(s_flags.loc["test"]).toarray())

In [134]:
pickle.dump(cv.transform(s_flags.loc["train"]).toarray(), open("../features/flags_train.pickle", "wb"))
pickle.dump(cv.transform(s_flags.loc["test"]).toarray(), open("../features/flags_test.pickle", "wb"))

# Content Features

In [111]:
pd_content_train = pd.read_csv("../results/content_tfidf_cv10k_linreg_count_date_img_twoyear_au_train.csv")
pd_content_test = pd.read_csv("../results/content_tfidf_cv10k_linreg_count_date_img_twoyear_au.csv")


In [112]:
d_features["content"] = (pd_content_train[ix_train].target.values.reshape(-1, 1),
                         pd_content_train[ix_test].target.values.reshape(-1, 1),
                         pd_content_test.target.values.reshape(-1, 1))

# Target

In [29]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [30]:
y_target = pd.read_csv("../data/train_target.csv")

In [31]:
y_target["key"] = "train"
y_target["ix"] = y_target.index.values

In [32]:
y_target.set_index(["key", "ix"], inplace=True)
y_target.index.names = [None, None]

# Модели

In [33]:
from sklearn.metrics import mean_absolute_error

In [34]:
from sklearn.ensemble import RandomForestRegressor

In [35]:
def get_RandomForestRegressor_n10():
    return RandomForestRegressor(n_estimators=10)
def get_RandomForestRegressor_n20():
    return RandomForestRegressor(n_estimators=20)

In [36]:
from sklearn.linear_model import LinearRegression

In [37]:
def get_LinearRegression():
    return LinearRegression(fit_intercept=True)

In [102]:
d_models = {
    # "01_LinearRegression": get_LinearRegression,
    "02_RandomForestRegressor_n10": get_RandomForestRegressor_n10,
    # "02_RandomForestRegressor_n20": get_RandomForestRegressor_n20,
    
}

# Modeling

In [39]:
feature_list = list(d_features.keys())
feature_list

['domains', 'robots', 'timestamp', 'lb_year']

In [40]:
Y_train = y_target[ix_train].target.values
Y_test = y_target[ix_test].target.values

In [41]:
def try_model(feature_list, reg):
    x_train = np.concatenate([d_features[k][0] for k in feature_list], axis=1)
    x_test = np.concatenate([d_features[k][1] for k in feature_list], axis=1)
    reg.fit(x_train, Y_train)
    Y_predict = reg.predict(x_test)
    return mean_absolute_error(Y_test, Y_predict)

# Запускаем перебор

In [42]:
iter_result = dict()

In [43]:
from tqdm import tqdm_notebook

In [104]:
import itertools

all_features = list(d_features.keys())
for model_name, model_func in tqdm_notebook(sorted(d_models.items()), desc="models", leave=False, disable=False):
    for L in tqdm_notebook(range(1, len(all_features)+1), desc="levels", leave=False, disable=False):
        for feature_list in tqdm_notebook(itertools.combinations(all_features, L), desc="features", leave=False, disable=False):
            if "content" not in feature_list:
                continue
            mae = try_model(feature_list, model_func())
            print(mae, model_name, feature_list)
            iter_result[(model_name, feature_list)] = mae

0.845205044705 02_RandomForestRegressor_n10 ('content',)


0.83353013359 02_RandomForestRegressor_n10 ('domains', 'content')


Exception in thread Thread-16:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/opt/conda/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



0.8307116055 02_RandomForestRegressor_n10 ('robots', 'content')
0.753533587792 02_RandomForestRegressor_n10 ('timestamp', 'content')
0.80421498981 02_RandomForestRegressor_n10 ('lb_year', 'content')
0.797259804065 02_RandomForestRegressor_n10 ('author_rating', 'content')
0.799800461625 02_RandomForestRegressor_n10 ('lb_month', 'content')
0.824977096624 02_RandomForestRegressor_n10 ('flags', 'content')


0.822621214387 02_RandomForestRegressor_n10 ('domains', 'robots', 'content')
0.734538400703 02_RandomForestRegressor_n10 ('domains', 'timestamp', 'content')
0.785409750816 02_RandomForestRegressor_n10 ('domains', 'lb_year', 'content')
0.78488339789 02_RandomForestRegressor_n10 ('domains', 'author_rating', 'content')
0.781231957454 02_RandomForestRegressor_n10 ('domains', 'lb_month', 'content')
0.811466598188 02_RandomForestRegressor_n10 ('domains', 'flags', 'content')
0.739183292767 02_RandomForestRegressor_n10 ('robots', 'timestamp', 'content')
0.794490907706 02_RandomForestRegressor_n10 ('robots', 'lb_year', 'content')
0.786066015737 02_RandomForestRegressor_n10 ('robots', 'author_rating', 'content')
0.782200988013 02_RandomForestRegressor_n10 ('robots', 'lb_month', 'content')
0.811121889493 02_RandomForestRegressor_n10 ('robots', 'flags', 'content')
0.750489747767 02_RandomForestRegressor_n10 ('timestamp', 'lb_year', 'content')
0.728059621156 02_RandomForestRegressor_n10 ('timestamp

0.727469675471 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'content')
0.782458042337 02_RandomForestRegressor_n10 ('domains', 'robots', 'lb_year', 'content')
0.77682317232 02_RandomForestRegressor_n10 ('domains', 'robots', 'author_rating', 'content')
0.774715208137 02_RandomForestRegressor_n10 ('domains', 'robots', 'lb_month', 'content')
0.799727387537 02_RandomForestRegressor_n10 ('domains', 'robots', 'flags', 'content')
0.741708656879 02_RandomForestRegressor_n10 ('domains', 'timestamp', 'lb_year', 'content')
0.727524147456 02_RandomForestRegressor_n10 ('domains', 'timestamp', 'author_rating', 'content')
0.736901690012 02_RandomForestRegressor_n10 ('domains', 'timestamp', 'lb_month', 'content')
0.728364051842 02_RandomForestRegressor_n10 ('domains', 'timestamp', 'flags', 'content')
0.751457347849 02_RandomForestRegressor_n10 ('domains', 'lb_year', 'author_rating', 'content')
0.754944999731 02_RandomForestRegressor_n10 ('domains', 'lb_year', 'lb_month', 'content')


0.729484886873 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'lb_year', 'content')
0.715401730979 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'author_rating', 'content')
0.733884616642 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'lb_month', 'content')
0.717631323192 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'flags', 'content')
0.746390735446 02_RandomForestRegressor_n10 ('domains', 'robots', 'lb_year', 'author_rating', 'content')
0.7418001401 02_RandomForestRegressor_n10 ('domains', 'robots', 'lb_year', 'lb_month', 'content')
0.762270401171 02_RandomForestRegressor_n10 ('domains', 'robots', 'lb_year', 'flags', 'content')
0.739868574691 02_RandomForestRegressor_n10 ('domains', 'robots', 'author_rating', 'lb_month', 'content')
0.761132598966 02_RandomForestRegressor_n10 ('domains', 'robots', 'author_rating', 'flags', 'content')
0.757638629123 02_RandomForestRegressor_n10 ('domains', 'robots', 'lb_month', 'fla

0.712588303404 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'lb_year', 'author_rating', 'content')
0.734458229744 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'lb_year', 'lb_month', 'content')
0.718338873569 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'lb_year', 'flags', 'content')
0.716768224854 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'author_rating', 'lb_month', 'content')
0.703347333196 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'author_rating', 'flags', 'content')
0.720138916171 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'lb_month', 'flags', 'content')
0.726408757223 02_RandomForestRegressor_n10 ('domains', 'robots', 'lb_year', 'author_rating', 'lb_month', 'content')
0.734199061633 02_RandomForestRegressor_n10 ('domains', 'robots', 'lb_year', 'author_rating', 'flags', 'content')
0.727956217603 02_RandomForestRegressor_n10 ('domains', 'robots', 'lb_year', 'lb_m

0.714033824421 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'lb_year', 'author_rating', 'lb_month', 'content')
0.707409945337 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'lb_year', 'author_rating', 'flags', 'content')
0.722137151911 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'lb_year', 'lb_month', 'flags', 'content')
0.702612553142 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'author_rating', 'lb_month', 'flags', 'content')
0.709009072036 02_RandomForestRegressor_n10 ('domains', 'robots', 'lb_year', 'author_rating', 'lb_month', 'flags', 'content')
0.716330711415 02_RandomForestRegressor_n10 ('domains', 'timestamp', 'lb_year', 'author_rating', 'lb_month', 'flags', 'content')
0.711029220417 02_RandomForestRegressor_n10 ('robots', 'timestamp', 'lb_year', 'author_rating', 'lb_month', 'flags', 'content')


0.707770547175 02_RandomForestRegressor_n10 ('domains', 'robots', 'timestamp', 'lb_year', 'author_rating', 'lb_month', 'flags', 'content')


# На лучшей модели

In [135]:
features = all_features
model = d_models["02_RandomForestRegressor_n10"]()

In [136]:
def get_final_prediction(feature_list, reg):
    x_train = np.concatenate([d_features[k][0] for k in feature_list], axis=1)
    x_test = np.concatenate([d_features[k][1] for k in feature_list], axis=1)
    reg.fit(x_train, Y_train)
    
    x_valid = np.concatenate([d_features[k][2] for k in feature_list], axis=1)
    return reg.predict(x_valid)

In [137]:
%%time
V_result = pd.read_json("../data/test.json", lines=True)[["_id"]].rename(columns={"_id": "url"})

CPU times: user 15.7 s, sys: 3.94 s, total: 19.6 s
Wall time: 19.6 s


In [138]:
V_result["target"] = get_final_prediction(features, model)

In [139]:
output_name = "../results/content_allfeatures_randomforest.csv"

In [140]:
V_result.to_csv(output_name, index=False)

In [141]:
!head $output_name

url,target
https://geektimes.ru/post/87455/,1.2955976999999999
https://geektimes.ru/post/87452/,0.0693147
https://geektimes.ru/post/87459/,1.3284481
https://habrahabr.ru/post/87461/,3.0358106
https://habrahabr.ru/post/5754/,0.42766660000000006
https://geektimes.ru/post/87460/,2.3569264
https://geektimes.ru/post/87462/,1.4905526
https://habrahabr.ru/post/87467/,3.0661112999999998
https://habrahabr.ru/post/87464/,4.053170999999999
